In [11]:
import weaviate
from dspy.retrieve.weaviate_rm import WeaviateRM
weaviate_recipes_index = WeaviateRM("WeaviateRecipesChunk",
                                    weaviate_client=weaviate.connect_to_local())

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_31172/1586068040.py:3: ResourceWarning: unclosed <socket.socket fd=77, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=6, laddr=('::1', 59640, 0, 0), raddr=('::1', 8080, 0, 0)>
  weaviate_recipes_index = WeaviateRM("WeaviateRecipesChunk",


In [12]:
import dspy

class AnswerQuestion(dspy.Signature):
    """Assess the context and answer the question"""

    question = dspy.InputField()
    context = dspy.InputField()
    answer = dspy.OutputField()

In [19]:
class LanguageProgram(dspy.Module):
    def __init__(self, retrieval_k=3):
        # Set desired LLM and your API Key as environment variables
        llm = "command-r-plus"
        api_key = "Ew13fNmR843lC8w1ZGDaqdkwH7kJlXdYgImZBXqx"
        self.available_llms = {
            "command-r-plus": lambda: dspy.Cohere(model="command-r-plus", api_key=api_key)
        } # save state for meta API
        if llm in self.available_llms:
            self.base_llm = self.available_llms[llm]()
        else:
            raise ValueError(f"Model {llm} is not available in RAG with Recipes.")
            
        # Connect to Weaviate
        self.weaviate_recipes_index = WeaviateRM("WeaviateRecipesChunk",
                                            weaviate_client=weaviate.connect_to_local())

        # Set DSPy defaults
        dspy.settings.configure(lm=self.base_llm, rm=self.weaviate_recipes_index)

        self.retrieve = dspy.Retrieve(k=retrieval_k)
        self.answer_question = dspy.ChainOfThought(AnswerQuestion)

    def get_meta(self):
        return self.available_llms

    def forward(self, chat_history):
        context = self.retrieve(chat_history).passages
        pred = self.answer_question(context=context, question=chat_history).answer
        return dspy.Prediction(answer=pred)

In [20]:
program = LanguageProgram()

In [24]:
program("How do I create a Weaviate schema with Cohere embeddings?")

Prediction(
    answer="Reasoning: Let's think step by step in order to produce the answer. We can break down the process into the following steps: \n1. Understand Weaviate and its schema creation process: Weaviate is a vector-based search engine that utilizes a schema to organize and search data. \n2. Identify the requirements for the schema: The schema should include the"
)

In [25]:
program.base_llm.inspect_history(n=1)




Assess the context and answer the question

---

Follow the following format.

Question: ${question}

Context: ${context}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Question: How do I create a Weaviate schema with Cohere embeddings?

Context:
[1] «Is my data in the Weaviate database lost?',\n",
       "  \"Are there any 'best practices' or guidelines to consider when designing a schema?\",\n",
       "  'Is it possible to create one-to-many relationships in the schema?',\n",
       "  'Do Weaviate classes have namespaces?',\n",
       "  'Are there restrictions on UUID formatting? Do I have to adhere to any standards?',\n",
       "  'If I do not specify a UUID during adding data objects, will Weaviate create one automatically?',\n",
       "  'Can I use Weaviate to create a traditional knowledge graph?',\n",
       "  'Why does Weaviate have a schema and not an ontology?',\n",
       "  'How can I retrieve the total objec

'\n\n\nAssess the context and answer the question\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nContext: ${context}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: How do I create a Weaviate schema with Cohere embeddings?\n\nContext:\n[1] «Is my data in the Weaviate database lost?\',\\n",\n       "  \\"Are there any \'best practices\' or guidelines to consider when designing a schema?\\",\\n",\n       "  \'Is it possible to create one-to-many relationships in the schema?\',\\n",\n       "  \'Do Weaviate classes have namespaces?\',\\n",\n       "  \'Are there restrictions on UUID formatting? Do I have to adhere to any standards?\',\\n",\n       "  \'If I do not specify a UUID during adding data objects, will Weaviate create one automatically?\',\\n",\n       "  \'Can I use Weaviate to create a traditional knowledge graph?\',\\n",\n       "  \'Why does Weaviate have a schema and not an ontolog